In [20]:
import pandas as pd
from tqdm import tqdm

In [21]:
elec_path='data/eaglei_data/'
storm_path='data/NOAA_StormEvents/'

In [22]:
eaglei_2015=pd.read_csv(elec_path+'eaglei_outages_2015.csv')
eaglei_2015

,fips_code,county,state,customers_out,run_start_time
0,1021,Chilton,Alabama,3,2015-01-01 00:00:00
1,5003,Ashley,Arkansas,3,2015-01-01 00:00:00
2,5023,Cleburne,Arkansas,1,2015-01-01 00:00:00
3,5085,Lonoke,Arkansas,2,2015-01-01 00:00:00
4,5107,Phillips,Arkansas,3,2015-01-01 00:00:00
...,...,...,...,...,...
12939152,55059,Kenosha,Wisconsin,1,2015-12-31 00:00:00
12939153,55079,Milwaukee,Wisconsin,1,2015-12-31 00:00:00
12939154,55097,Portage,Wisconsin,1,2015-12-31 00:00:00
12939155,55117,Sheboygan,Wisconsin,166,2015-12-31 00:00:00


In [23]:

def group_customers_out_by_date(df):
    """
    Group the DataFrame by county, state, and the date part of run_start_time, 
    and sum the customers_out values.

    Args:
        df (pd.DataFrame): Original data containing columns 
            ['fips_code', 'county', 'state', 'customers_out', 'run_start_time'].

    Returns:
        pd.DataFrame: Grouped data with columns 
            ['county', 'state', 'date', 'customers_out', 'run_start_time'].
    """
    # Make a copy of the DataFrame to avoid modifying the original
    df = df.copy()
    
    # Ensure 'run_start_time' is of datetime type
    df['run_start_time'] = pd.to_datetime(df['run_start_time'])

    # Extract the date part only
    df['date'] = df['run_start_time'].dt.date

    # Group by county, state, and date, and sum the customers_out
    grouped = df.groupby(['fips_code', 'date']).agg({
        'customers_out': 'sum'
    }).reset_index()

    # Add back a datetime version of the date if needed
    grouped['run_start_time'] = pd.to_datetime(grouped['date'])

    return grouped



In [24]:
df_example=group_customers_out_by_date(eaglei_2015)
df_example

,fips_code,date,customers_out,run_start_time
0,1001,2015-01-04,21,2015-01-04
1,1001,2015-01-05,2,2015-01-05
2,1001,2015-01-06,25,2015-01-06
3,1001,2015-01-08,24,2015-01-08
4,1001,2015-01-09,2,2015-01-09
...,...,...,...,...
485523,56029,2015-04-23,1950,2015-04-23
485524,56029,2015-04-24,10075,2015-04-24
485525,56029,2015-05-06,2275,2015-05-06
485526,56029,2015-05-16,5868,2015-05-16


In [25]:
Storms=pd.read_csv(storm_path+'StormEvents_details-ftp_v1.0_d2015_c20240716.csv')

In [26]:
def change_date_storm(df_storm):
    df_storm['BEGIN_DATE_TIME'] = pd.to_datetime(df_storm['BEGIN_DATE_TIME']).dt.date
    df_storm['END_DATE_TIME'] = pd.to_datetime(df_storm['END_DATE_TIME']).dt.date
    return df_storm
df_storm=change_date_storm(Storms)


/var/folders/02/n1hnl_j93_d4b92q_jxc8rz80000gn/T/ipykernel_61860/2862265501.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_storm['BEGIN_DATE_TIME'] = pd.to_datetime(df_storm['BEGIN_DATE_TIME']).dt.date
/var/folders/02/n1hnl_j93_d4b92q_jxc8rz80000gn/T/ipykernel_61860/2862265501.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_storm['END_DATE_TIME'] = pd.to_datetime(df_storm['END_DATE_TIME']).dt.date


In [27]:
def keep_c_storm(df_storm):
    df_storm = df_storm[df_storm['CZ_TYPE'] == 'C']
    return df_storm

def make_fips_storm(storms):
    storms['STATE_FIPS'] = storms['STATE_FIPS'].astype(str).str.zfill(2)
    storms['CZ_FIPS'] = storms['CZ_FIPS'].astype(str).str.zfill(3)
    storms['FIPS'] = storms['STATE_FIPS'] + storms['CZ_FIPS']
    return storms

df_storm=keep_c_storm(df_storm)
df_storm=make_fips_storm(df_storm)
df_storm

/var/folders/02/n1hnl_j93_d4b92q_jxc8rz80000gn/T/ipykernel_61860/2339681555.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  storms['STATE_FIPS'] = storms['STATE_FIPS'].astype(str).str.zfill(2)
/var/folders/02/n1hnl_j93_d4b92q_jxc8rz80000gn/T/ipykernel_61860/2339681555.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  storms['CZ_FIPS'] = storms['CZ_FIPS'].astype(str).str.zfill(3)
/var/folders/02/n1hnl_j93_d4b92q_jxc8rz80000gn/T/ipykernel_61860/2339681555.py:8: SettingWithCopyWarning: 
A value is trying

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE,FIPS
6,201509,5,1418,201509,5,1422,98915,594085,GEORGIA,13,...,NE,MAGRUDER,32.9400,-82.2000,32.9400,-82.2000,Some thunderstorms in the CSRA took down sever...,Sheriff reported trees down on Hickson Road.,CSV,13033
7,201509,10,1628,201509,10,1632,98918,594086,SOUTH CAROLINA,45,...,N,ELGIN,34.7100,-80.7200,34.7100,-80.7200,A few thunderstorms in the Eastern Midlands pr...,"Dispatch reported straight line wind damage, i...",CSV,45057
8,201509,4,1440,201509,4,1444,98915,594083,GEORGIA,13,...,ENE,MAGRUDER,32.9400,-82.1400,32.9400,-82.1400,Some thunderstorms in the CSRA took down sever...,Public reported a few small trees and large li...,CSV,13033
9,201509,5,1417,201509,5,1421,98915,594084,GEORGIA,13,...,ESE,KEYSVILLE,33.2100,-82.1500,33.2100,-82.1500,Some thunderstorms in the CSRA took down sever...,Sheriff reported trees down on George Perkins ...,CSV,13033
10,201509,10,1628,201509,10,1632,98918,594087,SOUTH CAROLINA,45,...,E,LANCASTER,34.7200,-80.7500,34.7200,-80.7500,A few thunderstorms in the Eastern Midlands pr...,Dispatch reported a large carport and shed blo...,CSV,45057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57901,201512,25,1525,201512,26,0,101375,609959,ALABAMA,01,...,S,HOLMES GAP,34.2331,-86.8490,34.2243,-86.8498,Numerous systems affecting the Tennessee valle...,Water completely covering the bridge over Lake...,CSV,01043
57902,201512,25,1035,201512,26,0,102024,610100,TENNESSEE,47,...,SE,WINCHESTER SPGS,35.2214,-86.1436,35.2131,-86.1441,Numerous systems affecting the Tennessee valle...,Eight inches of water over Old Estill Springs ...,CSV,47051
57903,201512,25,2215,201512,26,0,101375,609689,ALABAMA,01,...,ENE,CENTERVILLE,34.2835,-86.9144,34.2835,-86.9144,Numerous systems affecting the Tennessee valle...,Rainfall total as of 7:15pm observation: 10.79...,CSV,01043
57904,201512,23,1900,201512,23,1908,101367,606485,ALABAMA,01,...,WNW,MURPHY XRDS,34.9540,-88.0731,35.0060,-87.9580,A potent low pressure lifted from the central ...,A tornado touched down initially near the inte...,CSV,01077


In [28]:


# def update_customers_out(df_storm, df_example):
#     """
#     Update the 'customers_out' column in df_storm based on matching time intervals 
#     and FIPS codes from df_example.

#     Args:
#         df_storm (pd.DataFrame): DataFrame containing storm events with columns 
#             ['FIPS', 'BEGIN_DATE_TIME', 'END_DATE_TIME', ...].
#         df_example (pd.DataFrame): DataFrame containing customer outage events with columns 
#             ['fips_code', 'customers_out', 'run_start_time', ...].

#     Returns:
#         pd.DataFrame: Updated df_storm with the 'customers_out' column modified.
#     """

#     # Ensure correct data types
#     df_storm = df_storm.copy()
#     df_storm['FIPS'] = df_storm['FIPS'].astype('int64')
#     df_storm['BEGIN_DATE_TIME'] = pd.to_datetime(df_storm['BEGIN_DATE_TIME'])
#     df_storm['END_DATE_TIME'] = pd.to_datetime(df_storm['END_DATE_TIME'])
    
#     df_example = df_example.copy()
#     df_example['run_start_time'] = pd.to_datetime(df_example['run_start_time'])

#     # Initialize the customers_out column if it does not exist
#     if 'customers_out' not in df_storm.columns:
#         df_storm['customers_out'] = 0

#     # Iterate over each row in df_example
#     for idx, row in tqdm(df_example.iterrows(), total=len(df_example), desc="Updating customers_out"):
#         run_time = row['run_start_time']
#         customers_out_value = row['customers_out']
#         fips_code = row['fips_code']

#         # Create a mask to find matching storm records
#         mask = (
#             (df_storm['BEGIN_DATE_TIME'] <= run_time) & 
#             (df_storm['END_DATE_TIME'] >= run_time) & 
#             (df_storm['FIPS'] == fips_code)
#         )

#         # Add customers_out_value to matched rows
#         df_storm.loc[mask, 'customers_out'] += customers_out_value

#     return df_storm


def update_customers_out(df_storm, df_example):
    """
    Update the 'customers_out' column in df_storm based on matching time intervals 
    and FIPS codes from df_example, and store an array of run_start_time values 
    for matching outage events.

    Args:
        df_storm (pd.DataFrame): DataFrame containing storm events with columns 
            ['FIPS', 'BEGIN_DATE_TIME', 'END_DATE_TIME', ...].
        df_example (pd.DataFrame): DataFrame containing customer outage events with columns 
            ['fips_code', 'customers_out', 'run_start_time', ...].

    Returns:
        pd.DataFrame: Updated df_storm with 'customers_out' and 'run_start_times' columns modified.
    """

    # Ensure correct data types
    df_storm = df_storm.copy()
    df_storm['FIPS'] = df_storm['FIPS'].astype('int64')
    df_storm['BEGIN_DATE_TIME'] = pd.to_datetime(df_storm['BEGIN_DATE_TIME'])
    df_storm['END_DATE_TIME'] = pd.to_datetime(df_storm['END_DATE_TIME'])
    
    df_example = df_example.copy()
    df_example['run_start_time'] = pd.to_datetime(df_example['run_start_time'])
    df_example['fips_code'] = df_example['fips_code'].astype('int64')

    # Initialize columns if they do not exist
    if 'customers_out' not in df_storm.columns:
        df_storm['customers_out'] = 0
    if 'run_start_times' not in df_storm.columns:
        df_storm['run_start_times'] = [[] for _ in range(len(df_storm))]

    # Iterate over each row in df_example
    for idx, row in tqdm(df_example.iterrows(), total=len(df_example), desc="Updating customers_out"):
        run_time = row['run_start_time']
        customers_out_value = row['customers_out']
        fips_code = row['fips_code']

        # Create a mask to find matching storm records
        mask = (
            (df_storm['BEGIN_DATE_TIME'] <= run_time) & 
            (df_storm['END_DATE_TIME'] >= run_time) & 
            (df_storm['FIPS'] == fips_code)
        )

        # Update customers_out and append run_start_time to the list
        df_storm.loc[mask, 'customers_out'] += customers_out_value
        df_storm.loc[mask, 'run_start_times'] = df_storm.loc[mask, 'run_start_times'].apply(
            lambda x: x + [run_time]
        )

    return df_storm


df_storm=update_customers_out(df_storm, df_example)

Updating customers_out: 100%|██████████| 485528/485528 [05:21<00:00, 1510.92it/s]


In [43]:
import pandas as pd

def add_lead_time_column(df_storm):
    """
    Add a 'lead_time' column to df_storm, containing a list of time differences (in hours)
    between each run_start_time and BEGIN_DATE_TIME. If run_start_times is empty, set lead_time to 0.

    Args:
        df_storm (pd.DataFrame): DataFrame with columns ['BEGIN_DATE_TIME', 'run_start_times', ...].

    Returns:
        pd.DataFrame: Updated df_storm with a new 'lead_time' column.
    """
    # Ensure df_storm is a copy to avoid modifying the input
    df_storm = df_storm.copy()

    # Initialize the lead_time column
    df_storm['lead_time'] = df_storm.apply(
        lambda row: [
            (run_time - row['BEGIN_DATE_TIME']).total_seconds() / 86400  # Convert to hours
            for run_time in row['run_start_times']
        ] if row['run_start_times'] else [-1.0],
        axis=1
    )

    return df_storm

df_storm_updated = add_lead_time_column(df_storm)



In [44]:
df_storm_updated

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE,FIPS,customers_out,run_start_times,lead_time
6,201509,5,1418,201509,5,1422,98915,594085,GEORGIA,13,...,-82.2000,32.9400,-82.2000,Some thunderstorms in the CSRA took down sever...,Sheriff reported trees down on Hickson Road.,CSV,13033,271,[2015-09-05 00:00:00],[0.0]
7,201509,10,1628,201509,10,1632,98918,594086,SOUTH CAROLINA,45,...,-80.7200,34.7100,-80.7200,A few thunderstorms in the Eastern Midlands pr...,"Dispatch reported straight line wind damage, i...",CSV,45057,1397,[2015-09-10 00:00:00],[0.0]
8,201509,4,1440,201509,4,1444,98915,594083,GEORGIA,13,...,-82.1400,32.9400,-82.1400,Some thunderstorms in the CSRA took down sever...,Public reported a few small trees and large li...,CSV,13033,621,[2015-09-04 00:00:00],[0.0]
9,201509,5,1417,201509,5,1421,98915,594084,GEORGIA,13,...,-82.1500,33.2100,-82.1500,Some thunderstorms in the CSRA took down sever...,Sheriff reported trees down on George Perkins ...,CSV,13033,271,[2015-09-05 00:00:00],[0.0]
10,201509,10,1628,201509,10,1632,98918,594087,SOUTH CAROLINA,45,...,-80.7500,34.7200,-80.7500,A few thunderstorms in the Eastern Midlands pr...,Dispatch reported a large carport and shed blo...,CSV,45057,1397,[2015-09-10 00:00:00],[0.0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57901,201512,25,1525,201512,26,0,101375,609959,ALABAMA,01,...,-86.8490,34.2243,-86.8498,Numerous systems affecting the Tennessee valle...,Water completely covering the bridge over Lake...,CSV,1043,0,[],[-1.0]
57902,201512,25,1035,201512,26,0,102024,610100,TENNESSEE,47,...,-86.1436,35.2131,-86.1441,Numerous systems affecting the Tennessee valle...,Eight inches of water over Old Estill Springs ...,CSV,47051,0,[],[-1.0]
57903,201512,25,2215,201512,26,0,101375,609689,ALABAMA,01,...,-86.9144,34.2835,-86.9144,Numerous systems affecting the Tennessee valle...,Rainfall total as of 7:15pm observation: 10.79...,CSV,1043,0,[],[-1.0]
57904,201512,23,1900,201512,23,1908,101367,606485,ALABAMA,01,...,-88.0731,35.0060,-87.9580,A potent low pressure lifted from the central ...,A tornado touched down initially near the inte...,CSV,1077,0,[],[-1.0]


In [46]:
df_storm_updated.to_csv('leading_data.csv')

In [47]:
from sklearn.model_selection import train_test_split

# 假设你的 DataFrame 是 df_storm_updated
train_df, test_df = train_test_split(df_storm_updated, test_size=0.18, random_state=42)

train_df.to_csv('train_data.csv', index=False)
test_df.to_csv('test_data.csv', index=False)
